In [14]:
import os
import sys
import math
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from google.colab import drive
from sklearn import tree
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
def score_distanceM(Xobserve,Yobserve,Xprevu,Yprevu):
  X=(Xobserve-Xprevu)**2
  Y=(Yobserve-Yprevu)**2
  return np.mean(np.sqrt(X+Y))

In [16]:
cd /content/drive/My\ Drive/Colab\ Notebooks/

/content/drive/My Drive/Colab Notebooks


In [17]:
train=pd.read_csv('train_v1.csv')
validation=pd.read_csv('validation_v1.csv')
test=pd.read_csv('test_v1.csv')

In [18]:
feature_train=train[['nlos_indicator', 't1', 't2', 't3', 't4', 'skew', 'tof', 
                     'tof_skew','range', 'range_skew','loc_target_x', 'loc_target_y', 
                     'temperature_initiator','temperature_target', 'rssi_request', 
                     'rssi_ack', 'rssi_data','rssi_mean', 'label', 'orientation']]

xy_train=train[['loc_initiator_x','loc_initiator_y']]

feature_validation=validation[['nlos_indicator', 't1', 't2', 't3', 't4', 'skew', 'tof',
                               'tof_skew','range', 'range_skew','loc_target_x', 'loc_target_y',
                               'temperature_initiator','temperature_target', 'rssi_request', 
                               'rssi_ack', 'rssi_data','rssi_mean', 'label', 'orientation']]
xy_validation=validation[['loc_initiator_x','loc_initiator_y']]

feature_test=test[['nlos_indicator', 't1', 't2', 't3', 't4', 'skew', 'tof', 'tof_skew','range',
                   'range_skew','loc_target_x', 'loc_target_y', 'temperature_initiator','temperature_target',
                   'rssi_request', 'rssi_ack', 'rssi_data','rssi_mean', 'label', 'orientation']]

xy_test=test[['loc_initiator_x','loc_initiator_y']]

In [19]:
clf=tree.DecisionTreeRegressor()
clf= clf.fit(feature_train, xy_train)
print('score sur le test  :',clf.score(feature_test,xy_test))
print('score sur la validation  :',clf.score(feature_validation,xy_validation))

predict_test=clf.predict(feature_test)
predict_validation=clf.predict(feature_validation)
print()
print("RMSE sur le test: % f" %(mean_squared_error(predict_test,xy_test)))
print("RMSE sur la validation: % f" %(mean_squared_error(predict_validation,xy_validation)))
print()
print("MAE sur le test: % f" %(mean_absolute_error(predict_test,xy_test)))
print("MAE sur la validation: % f" %(mean_absolute_error(predict_validation,xy_validation)))
print()

print('score distance test : ',score_distanceM(xy_test.loc_initiator_x, xy_test.loc_initiator_y,predict_test[:,0],predict_test[:,1]))
print('score distance validation : ',score_distanceM(xy_validation.loc_initiator_x, xy_validation.loc_initiator_y,predict_validation[:,0],predict_validation[:,1]))

score sur le test  : 0.8524065870351366
score sur la validation  : 0.9587280448131816

RMSE sur le test:  0.309562
RMSE sur la validation:  0.080442

MAE sur le test:  0.349273
MAE sur la validation:  0.180505

score distance test :  0.5137949850805319
score distance validation :  0.2655287290740726


In [20]:
from sklearn.feature_selection import SequentialFeatureSelector
list_variable=['nlos_indicator', 't1', 't2', 't3', 't4', 'skew', 'tof', 'tof_skew','range',
                   'range_skew','loc_target_x', 'loc_target_y', 'temperature_initiator','temperature_target',
                   'rssi_request', 'rssi_ack', 'rssi_data','rssi_mean', 'label', 'orientation']

clf = tree.DecisionTreeRegressor()
sfs = SequentialFeatureSelector(clf, direction='backward',cv=5)
sfs.fit(feature_validation, xy_validation)
variable=[]
bool_choix=sfs.get_support()
for i in range(len(bool_choix)):
  if bool_choix[i]:
    variable.append(list_variable[i])
variable

['nlos_indicator',
 't3',
 't4',
 'skew',
 'tof',
 'tof_skew',
 'range_skew',
 'temperature_initiator',
 'rssi_data',
 'label']

In [21]:
feature_train=train[variable]

xy_train=train[['loc_initiator_x','loc_initiator_y']]

feature_validation=validation[variable]
xy_validation=validation[['loc_initiator_x','loc_initiator_y']]

feature_test=test[variable]

xy_test=test[['loc_initiator_x','loc_initiator_y']]
clf=tree.DecisionTreeRegressor()
clf= clf.fit(feature_train, xy_train)
print('score sur le test  :',clf.score(feature_test,xy_test))
print('score sur la validation  :',clf.score(feature_validation,xy_validation))

predict_test=clf.predict(feature_test)
predict_validation=clf.predict(feature_validation)
print()
print("RMSE sur le test: % f" %(mean_squared_error(predict_test,xy_test)))
print("RMSE sur la validation: % f" %(mean_squared_error(predict_validation,xy_validation)))
print()
print("MAE sur le test: % f" %(mean_absolute_error(predict_test,xy_test)))
print("MAE sur la validation: % f" %(mean_absolute_error(predict_validation,xy_validation)))
print()

print('score distance test : ',score_distanceM(xy_test.loc_initiator_x, xy_test.loc_initiator_y,predict_test[:,0],predict_test[:,1]))
print('score distance validation : ',score_distanceM(xy_validation.loc_initiator_x, xy_validation.loc_initiator_y,predict_validation[:,0],predict_validation[:,1]))

score sur le test  : 0.8001009700833979
score sur la validation  : 0.9649931379398269

RMSE sur le test:  0.419261
RMSE sur la validation:  0.068229

MAE sur le test:  0.420401
MAE sur la validation:  0.167125

score distance test :  0.6184269737620856
score distance validation :  0.2458457379057933


In [31]:
param_grid = {'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
              'max_features':['auto', 'sqrt', 'log2','None']
              }

# create an instance of GridSearchCV
grid_search = GridSearchCV(clf, param_grid, cv=5)

# fit the grid search to the data
grid_search.fit(feature_train, xy_train)

# print the best parameters
print("Best parameters found: ", grid_search.best_params_)
print("Best parameters score: ", grid_search.best_score_)

Best parameters found:  {'criterion': 'friedman_mse', 'max_features': 10}
Best parameters score:  0.9773958841235093


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/tree/_classes.py", line 1315, in fit
    super().fit(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/tree/_classes.py", line 178, in fit
    raise ValueError(
ValueError: Some value(s) of y are negative which is not allowed for Poisson regression.

  warn

In [27]:
feature_train=train[variable]

xy_train=train[['loc_initiator_x','loc_initiator_y']]

feature_validation=validation[variable]
xy_validation=validation[['loc_initiator_x','loc_initiator_y']]

feature_test=test[variable]

xy_test=test[['loc_initiator_x','loc_initiator_y']]
clf=tree.DecisionTreeRegressor(criterion='absolute_error',max_features='auto')
clf= clf.fit(feature_train, xy_train)
print('score sur le test  :',clf.score(feature_test,xy_test))
print('score sur la validation  :',clf.score(feature_validation,xy_validation))

predict_test=clf.predict(feature_test)
predict_validation=clf.predict(feature_validation)
print()
print("RMSE sur le test: % f" %(mean_squared_error(predict_test,xy_test)))
print("RMSE sur la validation: % f" %(mean_squared_error(predict_validation,xy_validation)))
print()
print("MAE sur le test: % f" %(mean_absolute_error(predict_test,xy_test)))
print("MAE sur la validation: % f" %(mean_absolute_error(predict_validation,xy_validation)))
print()

print('score distance test : ',score_distanceM(xy_test.loc_initiator_x, xy_test.loc_initiator_y,predict_test[:,0],predict_test[:,1]))
print('score distance validation : ',score_distanceM(xy_validation.loc_initiator_x, xy_validation.loc_initiator_y,predict_validation[:,0],predict_validation[:,1]))

score sur le test  : 0.8311624622391129
score sur la validation  : 0.9671490836684802

RMSE sur le test:  0.354124
RMSE sur la validation:  0.064028

MAE sur le test:  0.389788
MAE sur la validation:  0.164187

score distance test :  0.5733987472104711
score distance validation :  0.24152378119680923
